Data from: https://www.kaggle.com/datasets/whenamancodes/fraud-detection

In [1]:
import numpy as np
import pandas as pd

df_train = pd.read_csv("../data/creditcard_0_train.csv")
df_train

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,1387.0,-0.247635,0.367930,0.853269,-0.727001,-0.107909,-0.220377,1.064465,-0.000631,-0.751445,...,-0.268825,-1.040942,0.556685,0.004071,-0.876036,0.444537,-0.065031,0.024997,126.85,0
1,16632.0,1.125704,0.060368,0.693077,1.418097,-0.218836,0.469390,-0.461809,0.268230,1.777269,...,-0.385354,-0.789926,0.055323,-0.434480,0.339990,-0.528869,0.010824,0.000923,8.62,0
2,205.0,1.182399,0.059489,0.310673,0.828817,0.341209,1.112593,-0.292778,0.287394,0.268307,...,-0.337818,-0.776840,-0.056486,-1.356986,0.475790,-0.546519,0.063035,0.008047,12.99,0
3,5164.0,-0.915487,-3.399440,-1.126437,1.612170,-1.190341,-0.070311,1.488157,-0.434074,1.182271,...,0.487133,-0.819884,-1.135484,0.044055,0.218116,0.401155,-0.300773,0.175775,1129.10,0
4,3377.0,-0.347934,0.633134,0.541116,-2.526809,1.148422,0.087046,1.016614,-0.114287,0.546241,...,0.059253,0.594505,-0.456344,-1.315068,0.177886,-0.745462,0.254809,-0.039400,1.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9867,20658.0,1.061662,-0.465585,1.226568,0.253467,-0.753072,0.919268,-1.025993,0.427126,2.303447,...,-0.275893,-0.305272,0.136545,-0.275365,-0.086951,0.945988,-0.036805,-0.008126,20.00,0
9868,4949.0,1.031677,-0.006787,0.568099,1.583664,-0.348881,0.030754,-0.214084,0.147572,1.621096,...,-0.187458,-0.287500,-0.047609,0.114755,0.495479,-0.377993,-0.011818,0.003753,48.31,0
9869,5345.0,-0.914827,0.834317,2.130190,0.935591,-0.460889,0.029705,0.331374,-0.503894,1.797079,...,0.010262,0.640675,-0.307134,0.445520,-0.420362,0.451446,-0.660535,-0.044895,59.90,0
9870,654.0,-0.833568,0.606174,-0.051329,-2.091447,0.968764,-0.030220,0.887288,0.099009,0.834622,...,0.320782,1.239559,0.192074,-0.621025,-0.843584,-0.835690,0.164384,0.274361,9.90,0


In [2]:
df_train.groupby("Class").agg(count=("V1", "count"))

,count
Class,
0,9823
1,49


In [3]:
from imblearn.combine import SMOTEENN

X_train = df_train.drop(columns=["Time", "Class"])
y_train = df_train["Class"]
X_resampled, y_resampled = SMOTEENN(random_state=42).fit_resample(X_train, y_train)

X_resampled.shape, y_resampled.shape

((19497, 29), (19497,))

In [20]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier

param_grids = [
    {
        "name": "DecisionTreeClassifier", 
        "algorithm": DecisionTreeClassifier(), 
        "params": [{
            "criterion": ["gini", "entropy"],
            "splitter": ["best", "random"],
            "max_depth": [10, 50],
            "min_samples_split": [2, 10],
            "min_samples_leaf": [1, 4],
            "max_features": ["sqrt", "log2"],
            "random_state": [42]
        }]
    }, 
    {
        "name": "RandomForestClassifier",
        "algorithm": RandomForestClassifier(), 
        "params": [{
            "n_estimators": [100, 500],
            "max_depth": [10, 30],
            "min_samples_split": [2, 10],
            "min_samples_leaf": [1, 4],
            "bootstrap": [True, False],
            "criterion": ["gini", "entropy"]
        }]
    }, 
    {
        "name": "GradientBoostingClassifier",
        "algorithm": GradientBoostingClassifier(), 
        "params": [{
            "n_estimators": [50, 200], 
            "learning_rate": [0.01, 0.2], 
            "max_depth": [3, 5], 
            "min_samples_split": [2, 10], 
            "min_samples_leaf": [1, 4], 
            "subsample": [0.8, 1.0], 
            "max_features": ["sqrt", None]
        }]
    }, 
    {
        "name": "XGBClassifier",
        "algorithm": XGBClassifier(), 
        "params": [{
            "n_estimators": [100, 500],
            "learning_rate": [0.01, 0.2],
            "max_depth": [3, 6],
            "min_child_weight": [1, 10],
            "gamma": [0.5, 2],
            "subsample": [0.6, 1.0],
            "colsample_bytree": [0.6, 1.0],
            "reg_alpha": [0, 1], 
            "reg_lambda": [0, 1]
        }]
    }
]

In [ ]:
from itertools import product
from sklearn.model_selection import cross_validate
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:8080")
mlflow.set_experiment("grid_search_models")

for pg in param_grids:
    for p in pg["params"]:
        keys = p.keys()
        values = p.values()
        product_values = list(product(*values))

        for pv in product_values:
            grid = dict(zip(keys, pv))
            mdinst = pg["algorithm"].set_params(**grid)

            with mlflow.start_run():
                params = {"model_algorithm": pg["name"]}
                params.update(mdinst.get_params())
                mlflow.log_params(params)

                cv_results = cross_validate(
                    mdinst, X_resampled, y_resampled, 
                    scoring=["roc_auc", "accuracy", "precision", "recall", "f1"], 
                    cv=3
                )
                
                metrics = {k: v.mean() for k, v in cv_results.items()}
                mlflow.log_metrics(metrics)

2025/12/23 20:59:27 INFO mlflow.tracking.fluent: Experiment with name 'grid_search_models' does not exist. Creating a new experiment.


🏃 View run loud-ray-425 at: http://127.0.0.1:8080/#/experiments/1/runs/c3a565618e744790adbad946fba729ee
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/1
🏃 View run learned-shrimp-673 at: http://127.0.0.1:8080/#/experiments/1/runs/854e2343d6f34327a1c744f8f5295269
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/1
🏃 View run nosy-ant-440 at: http://127.0.0.1:8080/#/experiments/1/runs/cda02f3f670f4d33aa0287264f8db249
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/1
🏃 View run peaceful-gnu-674 at: http://127.0.0.1:8080/#/experiments/1/runs/3f6d9ded86454942906543e73a5dfcec
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/1
🏃 View run bright-newt-263 at: http://127.0.0.1:8080/#/experiments/1/runs/2117f6639dec4bd295e11696e4a2dbd5
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/1
🏃 View run bold-swan-397 at: http://127.0.0.1:8080/#/experiments/1/runs/019cd94abee741319d1773012c6c8c4f
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/1
🏃 

In [33]:
import ast

def auto_convert(s):
    try:
        value = ast.literal_eval(s)
        if isinstance(value, (int, float)):
            return value
        else:
            return s 
    except (ValueError, SyntaxError, TypeError):
        return s

df_run = mlflow.search_runs(experiment_ids=[1])
df_run_elg = df_run[df_run["metrics.test_roc_auc"] >= 0.8]
best_model = df_run_elg.sort_values(by="metrics.test_f1", ascending=False).reset_index(drop=True).iloc[0]
best_model_algorithm = best_model["params.model_algorithm"]
param_grid_best_model = [pg for pg in param_grids if pg["name"] == best_model_algorithm][0]

best_model_params = {
        k.split(".")[-1]: auto_convert(v) \
            for k, v in best_model[[f"params.{col}" for col in param_grid_best_model["params"][0].keys()]].to_dict().items()
    }

best_model = param_grid_best_model["algorithm"].set_params(**best_model_params) \
    .fit(X_resampled, y_resampled)

best_model

,"loss loss: {'log_loss', 'exponential'}, default='log_loss'The loss function to be optimized. 'log_loss' refers to binomial andmultinomial deviance, the same as used in logistic regression.It is a good choice for classification with probabilistic outputs.For loss 'exponential', gradient boosting recovers the AdaBoost algorithm.",'log_loss'
,"learning_rate learning_rate: float, default=0.1Learning rate shrinks the contribution of each tree by `learning_rate`.There is a trade-off between learning_rate and n_estimators.Values must be in the range `[0.0, inf)`.For an example of the effects of this parameter and its interaction with``subsample``, see:ref:`sphx_glr_auto_examples_ensemble_plot_gradient_boosting_regularization.py`.",0.2
,"n_estimators n_estimators: int, default=100The number of boosting stages to perform. Gradient boostingis fairly robust to over-fitting so a large number usuallyresults in better performance.Values must be in the range `[1, inf)`.",200
,"subsample subsample: float, default=1.0The fraction of samples to be used for fitting the individual baselearners. If smaller than 1.0 this results in Stochastic GradientBoosting. `subsample` interacts with the parameter `n_estimators`.Choosing `subsample < 1.0` leads to a reduction of varianceand an increase in bias.Values must be in the range `(0.0, 1.0]`.",0.8
,"criterion criterion: {'friedman_mse', 'squared_error'}, default='friedman_mse'The function to measure the quality of a split. Supported criteria are'friedman_mse' for the mean squared error with improvement score byFriedman, 'squared_error' for mean squared error. The default value of'friedman_mse' is generally the best as it can provide a betterapproximation in some cases... versionadded:: 0.18",'friedman_mse'
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, values must be in the range `[2, inf)`.- If float, values must be in the range `(0.0, 1.0]` and `min_samples_split` will be `ceil(min_samples_split * n_samples)`... versionchanged:: 0.18 Added float values for fractions.",10
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, values must be in the range `[1, inf)`.- If float, values must be in the range `(0.0, 1.0)` and `min_samples_leaf` will be `ceil(min_samples_leaf * n_samples)`... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.Values must be in the range `[0.0, 0.5]`.",0.0
,"max_depth max_depth: int or None, default=3Maximum depth of the individual regression estimators. The maximumdepth limits the number of nodes in the tree. Tune this parameterfor best performance; the best value depends on the interactionof the input variables. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.If int, values must be in the range `[1, inf)`.",3
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.Values must be in the range `[0.0, inf)`.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsamples at the current node, ``N_t_L`` is the number of samples in theleft child, and ``N_t_R`` is the number of samples in the right child.``N``, ``N_t``,

In [39]:
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score

mlflow.set_experiment("best_model_evaluation")

df_test = pd.read_csv("../data/creditcard_0_test.csv")
X_test = df_test.drop(columns=["Time", "Class"])
y_test = df_test["Class"]

with mlflow.start_run():
    params = {"model_algorithm": best_model_algorithm}
    params.update(best_model.get_params())
    mlflow.log_params(params)

    mlflow.sklearn.log_model(
        sk_model=best_model,
        artifact_path="best_model",
        input_example=X_test.iloc[[0]], 
        registered_model_name="creditcard_fraud_model"
    )

    y_pred = best_model.fit(X_resampled, y_resampled).predict(X_test)

    metrics = {
        "roc_auc": roc_auc_score(y_test, y_pred),
        "accuracy": accuracy_score(y_test, y_pred),
        "precision": precision_score(y_test, y_pred),
        "recall": recall_score(y_test, y_pred),
        "f1": f1_score(y_test, y_pred)
    }

    mlflow.log_metrics(metrics)

2025/12/23 23:16:16 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/home/champ/miniconda3/envs/auto-retrain-system/lib/python3.14/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
Successfully registered model 'creditcard_fraud_model'.
2025/12/23 23:16:20 INFO mlflow.store.model_regi

🏃 View run kindly-dog-217 at: http://127.0.0.1:8080/#/experiments/2/runs/b16cbe30c9a7435782c58066b99fddd1
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/2
